# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = '/root/autodl-tmp/ZhipuAI/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [6]:
input_text = "解释下乾卦是什么？"

In [7]:
response, history = base_model.chat(tokenizer, query=input_text)

In [8]:
print(response)

乾卦是八卦之一，也是八宫图之一，它的卦象是由三个阳爻夹一个阴爻构成，象征着天、阳气、刚强等含义。乾卦的卦辞是“元、亨、利、贞”，意味着“正、通、顺利、正统”。

乾卦象征着天、阳气、刚强、创造、领导等含义。在中医、道家、儒家等传统文化中，乾卦与震动、春天的气息、木、金属等有关，与春季、肝脏、 Hand（手）等器官也有关。乾卦与八卦中的“乾”字发音相同，也与其含义相关，代表天、刚强、创造等含义。

乾卦的五行属性是“木”，在五行中，它代表着春季、生长、勃发等含义，与树木、花草、青年等有关。乾卦也与其象征意义相关，代表创造、领导、刚强等含义。

乾卦在八宫图中的位置是北方，与事业、领导、运作等有关，也与心脏、头部、手等器官有关。乾卦的位置在八宫图中的北方，与事业、领导、运作等有关，也与心脏、头部、手等器官有关。


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [10]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，它的卦象是由两个阴爻夹一个阳爻构成，象征着矛盾、诉讼、争斗等含义。讼卦的卦辞是“元、亨、利、贞”，意味着“正、通、顺利、正统”。

讼卦象征着矛盾、诉讼、争斗等含义。在中医、道家、儒家等传统文化中，讼卦与土、黄色、法律等有关，与法律、诉讼、争斗等有关。讼卦也与其象征意义相关，代表矛盾、诉讼、争斗等含义。

讼卦的五行属性是“土”，在五行中，它代表着黄色、法律、诉讼等含义，与法律、诉讼、争斗等有关。讼卦也与其象征意义相关，代表矛盾、诉讼、争斗等含义。

讼卦在八宫图中的位置是西南，与法律、诉讼、争斗等有关，也与黄色、土、身体等有关。讼卦的位置在八宫图中的西南，与法律、诉讼、争斗等有关，也与黄色、土、身体等有关。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [11]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20250825_213807"

peft_model_path = f"models{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [12]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [13]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由两个乾卦叠加而成，代表天卦。它象征着天地的力量和存在着。在这个卦中，预示着天地的力量即将不合，预示着天地合一将再次实现。', 'content': '\n乾卦代表着大吉的卜问，预示着吉祥如意，顺其自然，无拘无束，无险无害。然而，过于刚健强求，会导致吉祥转变为凶险。乾卦象征着天地的力量，代表着刚健、刚强、坚贞、刚毅、刚健、刚烈、正直、刚正。\n\n在周易中，乾卦代表着大吉，预示着吉祥如意，顺其自然，无拘无束，无险无害。然而，过于刚健强求，会导致吉祥转变为凶险。乾卦象征着天地的力量，代表着刚健、刚强、坚贞、刚毅、刚健、刚烈、正直、刚正。\n\n乾卦的哲学内涵是：天行健，君子自强不息。君子应该学习天行健，即自强不息，不断自强，坚贞刚毅，刚健刚正，才能达到自强不息的境界。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250825_213807）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [20]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
{'name': '坎为云，云气上升，ESUPRIT 坎为云，云气上升，天地之间。地实然，云乃成，云之平路，地支也。后复后，行复行，云气乃舒，天布也。备地行，利见云，云乃行，反归其里。利见风，地乃广。利见雨，天乃和。利见节，地乃实。利见电，天乃破。利 hear 远，地乃舒。利 hear 近，地乃塞。利往见毛，地乃行。利去见伯，地乃止。 君子观此，取自用于止也。观于伤侧，伤侧乃止。观于喜神，喜神乃止。观于祥瑞，祥瑞乃止。观于异行，反余气，余气乃止。观于悖论，反余明，反余萌，余萌乃止。', 'content': 'tan(π/4)-20℃\n\ntan(π/4) = 1，此时气温为20℃。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250825_213807）：
[gMASK]sop 周易中的讼卦是什么? 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [19]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
{'name': '师卦，又称子卦，是由坎卦（水）和坤卦（地）相叠而成。在周易中，它是异卦而非卦，预示着吉祥。预示着吉祥的是否成立，取决于的解释。', 'content': '\n坎卦代表着云，象征着天下的流行，而坤卦象征着地上的顺从。子卦象征着柔顺的君子的德行，寓意着吉祥。然而，由于它是异卦而非卦，所以需要结合其他卦来解读。\n\n总体来说，子卦预示着吉祥，但需要确保行为合乎道德规范，才能获得吉祥。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20250825_213807）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [21]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [22]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，由六个阳爻（代表阳）组成，象征着刚健强劲的特性。在卜卦中，乾卦预示着先祖先人留下的德行和智慧，以及刚柔之力的统一，可以大事顺利，无往不利。同时，它也预示着虽然事业顺利，但需谨慎谨慎，因为过于刚健可能会引起众人的反感和对抗。在事业和生活中，我们需要保持谦逊、刚毅和谨慎的态度，以应对各种挑战。

在详细解释乾卦时，需要考虑到卦象的由來，以及卦象所代表的含义。卦象是由上卦和下卦组成，上卦为乾，下卦为坤。上卦的六爻全为阳，下卦的六爻全为阴。这种卦象象征着刚柔之力的统一，预示先祖先人留下的德行和智慧，以及刚柔之力的统一，可以大事顺利，无往不利。

在解卦时，需要考虑卦象的由來，以及卦象所代表的含义。乾卦的卦象是干燥的象形，代表着天，象征着刚健强劲的特性。在解卦时，需要结合卦象的象形和卦辞，以及所处的卦位，来理解乾卦的含义。

乾卦的卦辞为：“元、亨、利、贞”，意味着元、亨、利、贞的统一。元指代天，亨指代通，利指代顺利，贞指代正。因此，乾卦预示着先祖先人留下的德行和智慧，以及刚柔之力的统一，可以大事顺利，无往不利。

在乾卦中，每个爻都有特定的含义。阳爻代表刚强，阴爻代表柔弱。在卦象中，阳爻处于主导地位，代表着主导力量，而阴爻则处于从属地位，代表着 subordinate power。

在乾卦中，阳爻占多数，表示天宇广阔，云彩填满，预示先祖先人留下的德行和智慧，以及刚柔之力的统一，可以大事顺利，无往不利。在事业和生活中，我们需要保持谦逊、刚毅和谨慎的态度，以应对各种挑战。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 乾卦"是周易卦象中的一枚卦，由上卦乾（天）和下卦坤（地）组成，预示着吉利的变化。在卜筮时，上卦表示吉利，下卦表示吉祥。同时，双方相互依赖，相互支持，表示事情将会顺利。

在周易中，乾卦代表着天，象征着刚健、刚毅、运行不息的特点。它由六条直线组成，其中三条直线相互交叉，象征着天子的威严和权威。根据周易的解释，乾卦象征着刚健、刚毅、柔顺的态度，以及刚健的性能，可以得出结论，乾卦预示着吉祥的变


In [23]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是周易中的一卦，代表天地大水之象。它由两个卦组成，上卦是坤，下卦是坎。上卦为坤，意味着地广聚众，众相聚聚为坤，聚而成 earth（坤），表示地广聚众，众相聚聚的特性。下卦为坎，象征水流动，具有灵活变通的特点，流数为坎。因此，地水师卦象征着天地大水之象，反映了自然界的运行变化。

在周易中，卦象是用来指导人们行为和决策的符号，地水师卦也不例外。根据卦象，上下卦之间的相乘克作用需要等到 both 卦象都变成静止的坤（代表地）卦象时，才能达到和谐的状态。因此，地水师卦的寓意是，在共同的的目标下，要团结协作，互相谦让， Flexibility（灵活变通）和 adaptability（适应能力）是非常重要的。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表军队和指挥军情的卦象。根据《象辞》，这一卦象被解释为“地中有水”，象征着像大地一样包容和养育众人。根据《断易天机》，只有德高望重的长者来统率军队，才能获得吉祥无咎。


据北宋易学家邵雍解，得师卦者将面临困难重重，忧心劳众，宜包容别人，艰苦努力，摒除一切困难。台湾国学大儒傅佩荣解则提到，对于时运、财运、家宅和身体等方面会有相应影响。


传统解卦认为，师卦具有养兵聚众、出师攻伐之象，彼此有伤，难得安宁的大象。在运势方面，预示着困难重重，需要以正规行事，谨小慎微，严于律已。在事业、经商、求名、婚恋和决策等方面，都需要保持冷静、谨慎，注意避免敌人和困难带来的不利影响，必能成功。


In [24]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
{'name': '象：乾（天）动而观其象，则象为天，天行健，君子以自强不息。观卦象，观其志，损耗自行，得官运，利见大人。', 'content': '\n卦辞：初时不利，虽会其意，利观于色。后时则利，利见大人。\n\n卦象：损卦（利） 观卦（利） 卦象：观 利 利\n\n键词：观、天、健、元、利、大、志。\n\n观卦是乾卦（天）动而观其象，象为天，天行健，君子以自强不息。观卦象，观其志，损耗自行，得官运，利见大人。观卦预示着先会面临困难，需要观望形势，等待时机的到来。虽会其意，利观于色，但随后 times 则利，利见大人。观卦的运势会逐渐好转，有利于官运，有利于得到他人的帮助。\n\n观卦的运势特点为：初时不利，虽会其意，利观于色。后时则利，利见大人。运势会逐渐好转，有利于官运，有利于得到他人的帮助。\n\n观卦的综合作用是：初时不利，虽会其意，利观于色。后时则利，利见大人。'}


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么卦象？ 讼卦"是周易卦象中的一枚卦，由上卦坎（水）和下卦乾（天）组成，上卦为云，下卦为天。上卦坎为险，下卦乾为刚，象征着云浮聚于天上，待时降雨之象。讼卦象征着刚强之徒彼此相逼，两方不能相互听诉，且双方都欲刚直，不愿屈服，预示着双方 will 争强，最终导致诉讼。讼卦的特性是：云聚天际，风急雨疏，刚强互现。

讼卦的时运是：大象云聚天际，时运风急雨疏，刚强互现。

讼卦的运势：云聚天际，风急雨疏，刚强互现。

讼卦的运势：云聚天际，风急雨疏，刚强互现。

讼卦的运势：云聚天际，风急雨疏，刚强互现。

讼卦的运势：云聚天际，风急雨疏，刚强互现。

讼卦的运势：云聚天际，风急雨疏，刚强互现。

讼卦的运势：云聚天际，风急雨疏，刚强互现。

讼卦的运势：云聚天际，风急雨疏，刚强互现。

讼卦的运势：云聚天际，风急雨疏，刚强互现。

讼卦的运势：云聚天际，风急雨疏，刚强互现。

讼卦的运势：云聚天际，风急雨疏，刚强互现。

讼卦的运势：云聚天际，风急雨疏，刚强互现。

讼卦的运势：云聚天际，风急雨疏，刚强互现。

讼卦的运势：云聚天际，风急雨疏，刚强互现。

讼卦的运势：云聚天际，风急雨疏，刚强互现。

讼卦的运势：云聚天际，

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [25]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [37]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中六十四卦之一，也是八卦中的乾卦。乾卦象征着天，代表着刚强、力量、精神、创造力和领导力。乾卦是由两个阴爻夹一个阳爻构成，表示阳爻在阴爻之中，具有阳爻的特性，象征着天的刚强和力量。乾卦的卦象是由三个阳爻和一个阴爻构成，象征着阳爻的力量和精神，也象征着天的完整和和谐。

乾卦的卦辞是：“元、亨、利、贞”，表示天万物皆以此为尊，故亨；天精神合一，纯阳execv，故利；守道者，利普天下。”，“元”表示万物之始，“亨”表示通晓，发生，成功，“利”表示吉祥如意，“贞”表示正译。

乾卦的卦辞中，三个阳爻代表天，象征着阳，一个阴爻代表地，象征着阴。阴爻在阳爻之中，象征着天的精神，也象征着阴爻的和谐。乾卦的卦象既表现出阳刚之强，又保持阴柔之顺，具有和谐之致，故乾卦卦辞之“元、亨、利、贞”。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 解释下乾卦是什么？乾卦是《易经》中的第一卦，也是六十四卦中的首卦。乾卦象征着天，代表着刚强、壮丽、积极向上的精神，以及万物的生长繁衍。乾卦由两个阳爻夹一个阴爻构成，阴爻代表阴柔，阳爻代表阳刚，刚中有柔，柔中含刚，象征着天地的相辅相成。在《易经》中，乾卦的卦辞是“元”，即万物，表示宇宙间一切事物的生成。乾卦的卦象为天，表示阳刚之至，万物生长之象。在《易经》中，乾卦具有刚强、刚毅、积极向上的精神，以及象征着宇宙的完整性和统一性。


In [42]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦是什么

原始输出：
地水师卦（师卦）是《易经》中第五卦，由乾卦下，坤卦上，象征军队。师卦象征军队师出无名，兵者凶器，故需德行来修正，终归顺治，为吉。

师卦的卦象为乾为天，地为坤为地，表示军队师出无踪，师出无名，需德行修正。

师卦卦象上坤下乾，乾为师，坤为众，表示众师之师，众师之亲，得德行修正，顺服德，顺服天，大吉。

师卦卦象：乾 坤
师 师
名 亡
德 修
顺 承


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 地水师卦是什么? 师卦原文:师。贞,丈人吉,无咎。象曰:地中有水,师。君子以容民畜众。白话文解释:师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解:师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解:忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解:时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


In [39]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
讼卦（卦名：讼）是《易经》六十四卦之一，由上卦坎（水）下卦干（天）构成。坎卦为阴卦，乾卦为阳卦，阴卦 below阳卦，表示阴胜阳的状况。这种状况必然导致两个相争，所以卦名称为讼。

讼卦象征诉讼、诉讼、争议等。它告诉人们，处理争议应该以和平、协商的方式解决，避免诉讼，以免带来损失。

以下是《易经》中关于讼卦的卦辞：

卦名：讼
卦象：下乾上坎

卦辞：讼，贞（正）吉，永吉。
象：坎象征水，乾象征天；水天相包，顺服于天，详解：诉讼，得到永远的吉运。

初六：干复，显。
象：乾为阳，为天，为名，为声。象征乾卦下，坎卦上，宜公开、明显，明示自己的名份和地位。

六七：上下昆壮。
象：上下两个卦相合，象征刚强之势力。

初七：坤顺，贞；初八：坤拔，不吉。
象：坤为阴，坎为阳。坤象征阴，坎象征阳。阴不能胜阳，故运不吉。

八宫：坤宫。
象：坤卦下，坎卦上，为《易经》六宫图之一，象征顺利。

讼卦的卦辞和卦象都表明，坎卦象征阳，乾卦象征阴，阴不能胜阳，应该顺应阴阳平衡，才能获得好运。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
[gMASK]sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离不亲，慎谋退守则无凶。得此卦者，身心不安，多争诉，宜修身养性。台湾国学大儒傅佩荣解：时运受阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。
